In [1]:
# computing_rho_p

# Author: Christopher Keyes
# Updated 7 May 2024

# Sage file for paper titled "How often does a cubic hypersurface have a rational point?"
#     joint work with Lea Beneish
#     (Sage version 8.2)

In [2]:
# initialize variables:
#     p = prime (or prime power), symbolic
#     n = dimension of ambient projective space (can be symbolic, but it is a bit cleaner to run for each n)
p = var('p')
n = 5

# import time library, for timing computations
import time

In [3]:
# function to compute (decorated) xi values
#     Each xi_n,t^(c) is the probability that a cubic form in n+1 variables has certain factorization properties
#         "type" t = 0, 1, 2, 3 (1,2,3 correspond to triple line, star, triangle)
#         "condition (c) = 0, 1, 2, 3 (1,2,3 correspond to point, line, plane conditions)
#     See Section 3 of paper, namely Lemmas 3.4, 3.8
#
# INPUTS: n = dimension of ambient projective space
#         t = type (0 is NOT 1, 2, or 3)
#         c = condition (0 is no condition)
# OUTPUT: xi_n,t^(c)

def calc_xi(n,t,c):
    
    # compute number of coefficients
    Ncoeff = binomial(n+3,3)
    
    # compute in case of n=0
    if n==0:
        if c==0 or c==1: # these are the only conditions that make sense
            if t==1:     # ax^3 is automatically type 1, this is the only type that appears
                return 1
            else:
                return 0
        else:
            return 0
    
    # compute in case of n=1
    if n==1:
        if t==3 or c==3: # neither type 3 nor condition (3) occur for binary cubics
            return 0

        if t==1: # triple root
            if c==0:
                return 1/(p^2+1)

            if c==1:
                return 1/p^2

            if c==2:
                return 0

        if t==2: # irreducible binary cubic forms
            if c==0:
                return (p-1)*p/(3*(p^2+1))

            if c==1:
                return (p+1)*(p-1)/(3*p^2)

            if c==2:
                return 1

        if t==0:
            return 1 - calc_xi(n,1,c) - calc_xi(n,2,c) - calc_xi(n,3,c)

    # assume n at least 2
    # get type 0 by subtracting the others
    if t==0:
        return 1 - calc_xi(n,1,c) - calc_xi(n,2,c) - calc_xi(n,3,c)
    
    # from here on, assume t ¬= 0
    # no condition
    if c==0:
        Ntot = p^(Ncoeff) - 1

        if t==1: # triple line
            return (p^(n+1)-1)/Ntot

        if t==2: # star
            #return p*(p^(2*n+1) - p^(n+1) - p^n + 1)/(3*Ntot)
            Nn2 = 0
            for s in [0 .. n-1]:
                for t in [s+1 .. n]:
                    Nn2 = Nn2 + p^(t-s-1)*(p^3 - p)*p^(2*(n-t))
            Nn2 = Nn2*(p-1)/3
            return Nn2/Ntot

        if t==3: # triple line --- NOTE: BIT MESSY
            #Nn3 = (p*(p^3-p^2)/3)*((p^3*(p^(3*n-3)-1)/(p^3-1)) - (p*(p^(2*n-2)-1)/(p-1)) + (p^(n-1) - 1)/(p-1))
            Nn3 = 0
            for s in [0 .. n-2]:
                for t in [s+1 .. n-1]:
                    for u in [t+1 .. n]:
                        Nn3 = Nn3 + p^(t-s-1)*(p^3-p)*p^(2*(u-t-1))*(p^3-p^2)*p^(3*(n-u))
            Nn3 = Nn3*(p-1)/3            
            return Nn3/Ntot

    # condition c rules out types t<c
    if t < c:
        return 0
    
    # from here on, assume t>=c
    # point condition
    if c==1:
        Ntot = (p-1)*(p^(Ncoeff - 1))

        if t==1: # triple line
            return (p-1)*p^n/Ntot

        if t==2: # star
            #return (p^n*(p^2-1)*(p^n-1))/(3*Ntot)
            Nn2 = 0
            for t in [1 .. n]:
                Nn2 = Nn2 + p^(t-1)*(p^3 - p)*p^(2*(n-t))
            Nn2 = Nn2*(p-1)/3
            return Nn2/Ntot

        if t==3: # triangle
            #return (p^(n+1)*(p-1)*(p^(2*n-1)-p^n-p^(n-1) + 1))/(3*Ntot)
            Nn3 = 0
            for t in [1 .. n-1]:
                for u in [t+1 .. n]:
                    Nn3 = Nn3 + p^(t-1)*(p^3 - p)*p^(2*(u-t-1))*(p^3 - p^2)*p^(3*(n-u))
            Nn3 = Nn3*(p-1)/3
            return Nn3/Ntot

    # line condition
    if c==2:
        Ntot = (p-1)^2*(p+1)*p^(Ncoeff-3)/3

        if t==2: # star
            return ((p-1)*(p^3-p)*p^(2*n-2))/(3*Ntot)

        if t==3: # triangle
            #return (p^(2*n-1)*(p-1)*(p^2-1)*(p^(n-1)-1))/(3*Ntot)
            Nn3 = 0
            for u in [2 .. n]:
                Nn3 = Nn3 + (p^3 - p)*p^(2*(u-2))*(p^3 - p^2)*p^(3*(n-u))
            Nn3 = Nn3*(p-1)/3
            return Nn3/Ntot

    # plane condition
    if c==3:
        Ntot = (p-1)^3*(p+1)*p^(Ncoeff-7)/3

        if t==3: # triangle
            return (p^(3*n-3)*(p-1)^3*(p+1))/(3*Ntot)

In [4]:
# Compute values of theta_{n,ijk} --- see Phase III

# Start clock
time_start = time.time()

# Define thetas symbolically and assemble into array for easy indexing
th_111, th_112, th_113, th_121, th_122, th_123, th_131, th_132, th_133, th_211, th_212, th_213, th_221, th_222, th_223, th_231, th_232, th_233, th_311, th_312, th_313, th_321, th_322, th_323, th_331, th_332, th_333 = var('th_111, th_112, th_113, th_121, th_122, th_123, th_131, th_132, th_133, th_211, th_212, th_213, th_221, th_222, th_223, th_231, th_232, th_233, th_311, th_312, th_313, th_321, th_322, th_323, th_331, th_332, th_333') # thetas
Thetas = [th_111, th_112, th_113, th_121, th_122, th_123, th_131, th_132, th_133, th_211, th_212, th_213, th_221, th_222, th_223, th_231, th_232, th_233, th_311, th_312, th_313, th_321, th_322, th_323, th_331, th_332, th_333]

# get_theta: returns theta variable
#     INPUTS: i,j,k in {1,2,3}
#     OUTPUT: theta_ijk
def get_theta(i,j,k):
    return Thetas[9*(i-1) + 3*(j-1) + (k-1)]

# theta_rels: returns list of relations defining theta values
#     INPUTS:  n = dimension of ambient projective space
#     OUTPUTS: list of 27 relations
def theta_rels(n):
    L = []
    
    for i in [1 .. 3]:
        for j in [1 .. 3]:
            for k in [1 .. 3]:
                right_side = 0
                
                if i + j + k < n + 1: # if i+j+k >= n+1 there can be no solution, return 0
                    right_side = right_side + calc_xi(n-j-k,0,i)
                    for l in [1 .. 3]:
                        right_side = right_side + calc_xi(n-j-k,l,i)*get_theta(j,k,l)
                        
                    right_side = right_side/p^(i*j*(n-i-j-k+1) + j*binomial(n-i-j-k+2,2)) + (1 - 1/p^(i*j*(n-i-j-k+1) + j*binomial(n-i-j-k+2,2)))
                    
                L.append((get_theta(i,j,k) == right_side))
                
    return L

# Solve for thetas and record values
Stheta = solve(theta_rels(n), Thetas)

Theta_vals = []
for i in range(0, len(Thetas)):
    Theta_vals.append(Stheta[0][i].right())

# get_theta_val: returns theta value
#     returns theta_ijk
def get_theta_val(i,j,k):
    return Theta_vals[9*(i-1) + 3*(j-1) + (k-1)]

# stop clock
theta_time = time.time() - time_start

In [5]:
# Compute values of tau_{n,ij}, tau'_{n,ij}, and sigma_{n,i}^{(c)} --- see Phases I and II

# Start clock
time_start = time.time()

# Define probabilities symbolically and assemble into arrays for easy indexing
s_1_1, s_2_1, s_3_1, s_1_2, s_2_2, s_3_2, s_1_3, s_2_3, s_3_3 = var('s_1_1, s_2_1, s_3_1, s_1_2, s_2_2, s_3_2, s_1_3, s_2_3, s_3_3') # sigmas with conditions
t_11, t_12, t_13, t_21, t_22, t_23, t_31, t_32, t_33 = var('t_11, t_12, t_13, t_21, t_22, t_23, t_31, t_32, t_33') # taus
t_11_prime, t_12_prime, t_13_prime, t_21_prime, t_22_prime, t_23_prime, t_31_prime, t_32_prime, t_33_prime = var('t_11_prime, t_12_prime, t_13_prime, t_21_prime, t_22_prime, t_23_prime, t_31_prime, t_32_prime, t_33_prime') # tau primes
Sigma_cond  = [s_1_1, s_2_1, s_3_1, s_1_2, s_2_2, s_3_2, s_1_3, s_2_3, s_3_3]
Taus        = [t_11, t_12, t_13, t_21, t_22, t_23, t_31, t_32, t_33]
Tau_primes  = [t_11_prime, t_12_prime, t_13_prime, t_21_prime, t_22_prime, t_23_prime, t_31_prime, t_32_prime, t_33_prime]

# get_sigma_cond: returns sigma variable (not sigma_prime)
#     i determines the type (i=1,2,3)
#     j determines the condition (j=1,2,3)
def get_sigma_cond(i,j):
    return Sigma_cond[i-1 + 3*(j-1)]
    
# get_tau: returns tau variable (not tau_prime)
#     returns tau_ij
def get_tau(i,j):
    return Taus[3*(i-1) + (j-1)]

# get_tau_prime: returns tau_prime variable 
#     returns tau_ij_prime
def get_tau_prime(i,j):
    return Tau_primes[3*(i-1) + (j-1)]

# sigma_cond_rels: returns list of relations defining sigma values
#     INPUTS:  n = dimension of ambient projective space
#     OUTPUTS: list of 9 relations
def sigma_cond_rels(n):
    L = []            

    for k in [1 .. 3]:
        for i in [1 .. 3]:
            if i > n: # only relevant if n=2
                L.append((get_sigma_cond(i,k) == 0))
                
            else: 
                right_side = calc_xi(n-i,0,k)
                for j in [1 .. 3]:
                    right_side = right_side + calc_xi(n-i,j,k)*get_tau(i,j)

                L.append((get_sigma_cond(i,k) == right_side))
            
    return L

# tau_rels: returns list of relations defining tau values
#     INPUTS:  n = dimension of ambient projective space
#     OUTPUTS: list of 9 relations
def tau_rels(n):
    L = []
    
    for i in [1 .. 3]:
        for j in [1 .. 3]:
            right_side = 0
            
            if i + j < n + 1: # if i+j >= n+1, then there can be no solution, return 0
                right_side = right_side + calc_xi(n-i-j,0,0)
                for k in [1 .. 3]:
                    right_side = right_side + calc_xi(n-i-j,k,0)*get_theta_val(i,j,k)

                right_side = right_side*(1-1/p^(binomial(n-i-j+3,3)))
                right_side = right_side + get_tau_prime(i,j)/p^(binomial(n-i-j+3,3))
                right_side = right_side/p^(i*binomial(n-i-j+2,2)) + (1 - 1/p^(i*binomial(n-i-j+2,2)))

            L.append((get_tau(i,j) == right_side))
            
    return L

# tau_prime_rels: returns list of relations defining tau_prime values
#     INPUTS:  n = dimension of ambient projective space
#     OUTPUTS: list of 9 relations
def tau_prime_rels(n):
    L = []
    
    for i in [1 .. 3]:
        for j in [1 .. 3]:
            right_side = 0
            
            if i + j < n + 1: # if i+j >= n+1, then there can be no solution, return 0
                right_side = right_side + calc_xi(n-j,0,i)
                for k in [1 .. 3]:
                    right_side = right_side + calc_xi(n-j,k,i)*get_sigma_cond(k,j)
                    
                right_side = right_side/p^(i*j*(n-i-j+1) + j*binomial(n-i-j+2,2)) + (1 - 1/p^(i*j*(n-i-j+1) + j*binomial(n-i-j+2,2)))

            L.append((get_tau_prime(i,j) == right_side))
            
    return L

# Solve system of equations
Lsigma_cond = sigma_cond_rels(n)
Ltau = tau_rels(n)
Ltau_prime = tau_prime_rels(n)
L2 = Lsigma_cond + Ltau + Ltau_prime
Vars2 = Sigma_cond + Taus + Tau_primes
S2 = solve(L2, Vars2)

# Store values in arrays
Sigma_cond_vals = []
Tau_vals = []
Tau_prime_vals = []
for i in [0 .. 8]:
    Sigma_cond_vals.append(S2[0][i].right())
for i in [9 .. 17]:
    Tau_vals.append(S2[0][i].right())
for i in [18 .. 26]:
    Tau_prime_vals.append(S2[0][i].right())

# get_sigma_cond_val: returns sigma value (not sigma_prime)
#     i determines the type (i=1,2,3)
#     j determines the condition (j=1,2,3)
def get_sigma_cond_val(i,j):
    return Sigma_cond_vals[i-1 + 3*(j-1)]
    
# get_tau_val: returns tau value (not tau_prime)
#     returns tau_ij
def get_tau_val(i,j):
    return Tau_vals[3*(i-1) + (j-1)]

# get_tau_prime_val: returns tau_prime value 
#     returns tau_ij_prime
def get_tau_prime_val(i,j):
    return Tau_prime_vals[3*(i-1) + (j-1)]

# stop clock
tau_time = time.time() - time_start

In [6]:
# Final step: compute rhos, sigmas, sigma_primes --- see Phase I

# start clock
time_start = time.time()

# define variables for rho, sigma, sigma' and assemble into array for easy indexing
r, r_1, r_2, r_3 = var('r, r_1, r_2, r_3') 
s_1, s_2, s_3, s_1_prime, s_2_prime, s_3_prime = var('s_1, s_2, s_3, s_1_prime, s_2_prime, s_3_prime') 
Rhos         = [r, r_1, r_2, r_3]
Sigmas       = [s_1, s_2, s_3]
Sigma_primes = [s_1_prime, s_2_prime, s_3_prime]

# get_rho: returns rho variable
#     i=0 returns rho itself
#     i=1,2,3 returns rho^(i)
def get_rho(i):
    return Rhos[i]

# get_sigma: returns sigma variable (not sigma_prime, no condition)
#     i determines the type (i=1,2,3)
def get_sigma(i):
    return Sigmas[i-1]

# get_sigma_prime: returns sigma_prime variable
#     i determines the type (i=1,2,3)
def get_sigma_prime(i):
    return Sigma_primes[i-1]

# rho_rels: returns list of relations defining rho values
#     INPUTS:  n = dimension of ambient projective space
#     OUTPUTS: list of 4 relations
def rho_rels(n):
    L = []
    
    for j in [0 .. 3]:
        right_side = calc_xi(n,0,j)
        for i in [1 .. 3]:
            right_side = right_side + calc_xi(n,i,j)*get_sigma(i)
        
        L.append((get_rho(j) == right_side))
    
    return L

# sigma_rels: returns list of relations defining sigma values
#     INPUTS:  n = dimension of ambient projective space
#     OUTPUTS: list of 3 relations
def sigma_rels(n):
    L = []
    
    for i in [1 .. 3]:
        if i > n: # only relevant if n=2
            L.append((get_sigma(i) == 0))
            
        else:
            right_side = calc_xi(n-i,0,0)
            for j in [1 .. 3]:
                right_side = right_side + calc_xi(n-i,j,0)*get_tau_val(i,j)

            right_side = right_side*(1 - 1/p^(binomial(n-i+3,3)))
            right_side = right_side + (1/p^(binomial(n-i+3,3)))*get_sigma_prime(i)

            L.append((get_sigma(i) == right_side))
    
    return L
    
# sigma_prime_rels: returns list of relations defining sigma_prime values
#     INPUTS:  n = dimension of ambient projective space
#     OUTPUTS: list of 3 relations
def sigma_prime_rels(n):
    L = []
    
    for i in [1 .. 3]:
        if i > n: # only relevant if n=2
            L.append((get_sigma_prime(i) == 0))
            
        else:
            right_side = calc_xi(n-i,0,0)

            for j in [1 .. 3]:
                right_side = right_side + calc_xi(n-i,j,0)*get_sigma_cond_val(j,i)

            right_side = right_side*(1-1/p^(binomial(n-i+3,3)))
            right_side = right_side + get_rho(i)/p^(binomial(n-i+3,3))
            right_side = right_side/p^(i*binomial(n-i+2,2)) + (1 - 1/p^(i*binomial(n-i+2,2)))

            L.append((get_sigma_prime(i) == right_side))
    
    return L

# solve system
Lrho = rho_rels(n)
Lsigma = sigma_rels(n)
Lsigma_prime = sigma_prime_rels(n)
L3 = Lrho + Lsigma + Lsigma_prime
Vars3 = Rhos + Sigmas + Sigma_primes
S3 = solve(L3, Vars3)

# record values
Rho_vals = []
Sigma_vals = []
Sigma_prime_vals = []

for i in [0 .. 3]:
    Rho_vals.append(S3[0][i].right())
for i in [4 .. 6]:
    Sigma_vals.append(S3[0][i].right())
for i in [7 .. 9]:
    Sigma_prime_vals.append(S3[0][i].right())
    
# get_rho_val: returns rho value
#     i=0 returns rho itself
#     i=1,2,3 returns rho^(i)
def get_rho_val(i):
    return Rho_vals[i]

# get_sigma_val: returns sigma value (not sigma_prime, no condition)
#     i determines the type (i=1,2,3)
def get_sigma_val(i):
    return Sigma_vals[i-1]

# get_sigma_prime_val: returns sigma_prime value
#     i determines the type (i=1,2,3)
def get_sigma_prime_val(i):
    return Sigma_prime_vals[i-1]

# end clock
rho_time = time.time() - time_start

In [7]:
# printing results
rho = get_rho_val(0)

print 'n =', n
print 'time elapsed:', theta_time + tau_time + rho_time
print ''
print 'rho =', rho
print ''
print '1-rho = g/h:'
print ''
print 'g =', (1-rho).numerator()
print ''
print 'h =', (1-rho).denominator()

# NOTE - these values of g, h might differ slightly from those presented in the paper.
#     The ratio of g/h (i.e. 1-rho) is the same. This is due to factoring the denominator h,
#     and adding certain factors to numerator and denominator to produce g, h which fit
#     more neatly on the page in the paper.

n = 5
time elapsed: 28.3097250462

rho = 1/27*(27*p^134 + 54*p^133 + 108*p^132 + 162*p^131 + 270*p^130 + 405*p^129 + 621*p^128 + 864*p^127 + 1215*p^126 + 1620*p^125 + 2187*p^124 + 2835*p^123 + 3699*p^122 + 4671*p^121 + 5913*p^120 + 7317*p^119 + 9072*p^118 + 11043*p^117 + 13446*p^116 + 16119*p^115 + 19332*p^114 + 22896*p^113 + 27108*p^112 + 31725*p^111 + 37098*p^110 + 42957*p^109 + 49680*p^108 + 56943*p^107 + 65151*p^106 + 73953*p^105 + 83808*p^104 + 94284*p^103 + 105867*p^102 + 118071*p^101 + 131436*p^100 + 145422*p^99 + 160596*p^98 + 176337*p^97 + 193239*p^96 + 210627*p^95 + 229149*p^94 + 248049*p^93 + 268002*p^92 + 288141*p^91 + 309207*p^90 + 330264*p^89 + 352077*p^88 + 373653*p^87 + 395766*p^86 + 417363*p^85 + 439293*p^84 + 460431*p^83 + 481650*p^82 + 501798*p^81 + 521775*p^80 + 540402*p^79 + 558627*p^78 + 575235*p^77 + 591189*p^76 + 605289*p^75 + 618543*p^74 + 629718*p^73 + 639875*p^72 + 647756*p^71 + 654476*p^70 + 658830*p^69 + 661932*p^68 + 662577*p^67 + 661935*p^66 + 658827*p^65

In [8]:
# recorded values of rho(p) (computed 18 March 2024)
rho1 = 1/3*(2*p^4 + 3*p^3 + p^2 + 3*p + 2)/(p^4 + p^3 + p^2 + p + 1)
rho2 = 1/3*(3*p^12 + 3*p^10 + 2*p^9 + 4*p^8 + 3*p^7 + 5*p^6 + 3*p^5 + 4*p^4 + 2*p^3 + 2*p^2 + 2*p + 2)/(p^12 + p^10 + p^9 + p^8 + p^7 + 2*p^6 + p^5 + p^4 + p^3 + p^2 + 1)
rho3 = 1/9*(9*p^36 + 18*p^35 + 36*p^34 + 54*p^33 + 90*p^32 + 117*p^31 + 162*p^30 + 198*p^29 + 261*p^28 + 306*p^27 + 375*p^26 + 432*p^25 + 512*p^24 + 557*p^23 + 617*p^22 + 651*p^21 + 690*p^20 + 691*p^19 + 709*p^18 + 694*p^17 + 693*p^16 + 648*p^15 + 620*p^14 + 557*p^13 + 515*p^12 + 429*p^11 + 378*p^10 + 306*p^9 + 261*p^8 + 195*p^7 + 162*p^6 + 117*p^5 + 90*p^4 + 54*p^3 + 36*p^2 + 18*p + 9)/(p^36 + 2*p^35 + 4*p^34 + 6*p^33 + 10*p^32 + 13*p^31 + 18*p^30 + 22*p^29 + 29*p^28 + 34*p^27 + 42*p^26 + 48*p^25 + 57*p^24 + 62*p^23 + 69*p^22 + 72*p^21 + 77*p^20 + 77*p^19 + 79*p^18 + 77*p^17 + 77*p^16 + 72*p^15 + 69*p^14 + 62*p^13 + 57*p^12 + 48*p^11 + 42*p^10 + 34*p^9 + 29*p^8 + 22*p^7 + 18*p^6 + 13*p^5 + 10*p^4 + 6*p^3 + 4*p^2 + 2*p + 1)
rho4 = 1/9*(9*p^68 + 18*p^67 + 36*p^66 + 54*p^65 + 90*p^64 + 135*p^63 + 198*p^62 + 261*p^61 + 351*p^60 + 450*p^59 + 585*p^58 + 720*p^57 + 891*p^56 + 1071*p^55 + 1296*p^54 + 1530*p^53 + 1809*p^52 + 2088*p^51 + 2412*p^50 + 2727*p^49 + 3087*p^48 + 3429*p^47 + 3797*p^46 + 4131*p^45 + 4491*p^44 + 4815*p^43 + 5157*p^42 + 5433*p^41 + 5723*p^40 + 5950*p^39 + 6183*p^38 + 6335*p^37 + 6479*p^36 + 6533*p^35 + 6582*p^34 + 6534*p^33 + 6480*p^32 + 6336*p^31 + 6183*p^30 + 5949*p^29 + 5724*p^28 + 5433*p^27 + 5158*p^26 + 4814*p^25 + 4491*p^24 + 4131*p^23 + 3798*p^22 + 3429*p^21 + 3087*p^20 + 2726*p^19 + 2412*p^18 + 2088*p^17 + 1809*p^16 + 1530*p^15 + 1296*p^14 + 1071*p^13 + 891*p^12 + 720*p^11 + 585*p^10 + 450*p^9 + 351*p^8 + 261*p^7 + 198*p^6 + 135*p^5 + 90*p^4 + 54*p^3 + 36*p^2 + 18*p + 9)/(p^68 + 2*p^67 + 4*p^66 + 6*p^65 + 10*p^64 + 15*p^63 + 22*p^62 + 29*p^61 + 39*p^60 + 50*p^59 + 65*p^58 + 80*p^57 + 99*p^56 + 119*p^55 + 144*p^54 + 170*p^53 + 201*p^52 + 232*p^51 + 268*p^50 + 303*p^49 + 343*p^48 + 381*p^47 + 422*p^46 + 459*p^45 + 499*p^44 + 535*p^43 + 573*p^42 + 604*p^41 + 636*p^40 + 661*p^39 + 687*p^38 + 704*p^37 + 720*p^36 + 726*p^35 + 731*p^34 + 726*p^33 + 720*p^32 + 704*p^31 + 687*p^30 + 661*p^29 + 636*p^28 + 604*p^27 + 573*p^26 + 535*p^25 + 499*p^24 + 459*p^23 + 422*p^22 + 381*p^21 + 343*p^20 + 303*p^19 + 268*p^18 + 232*p^17 + 201*p^16 + 170*p^15 + 144*p^14 + 119*p^13 + 99*p^12 + 80*p^11 + 65*p^10 + 50*p^9 + 39*p^8 + 29*p^7 + 22*p^6 + 15*p^5 + 10*p^4 + 6*p^3 + 4*p^2 + 2*p + 1)
rho5 = 1/27*(27*p^134 + 54*p^133 + 108*p^132 + 162*p^131 + 270*p^130 + 405*p^129 + 621*p^128 + 864*p^127 + 1215*p^126 + 1620*p^125 + 2187*p^124 + 2835*p^123 + 3699*p^122 + 4671*p^121 + 5913*p^120 + 7317*p^119 + 9072*p^118 + 11043*p^117 + 13446*p^116 + 16119*p^115 + 19332*p^114 + 22896*p^113 + 27108*p^112 + 31725*p^111 + 37098*p^110 + 42957*p^109 + 49680*p^108 + 56943*p^107 + 65151*p^106 + 73953*p^105 + 83808*p^104 + 94284*p^103 + 105867*p^102 + 118071*p^101 + 131436*p^100 + 145422*p^99 + 160596*p^98 + 176337*p^97 + 193239*p^96 + 210627*p^95 + 229149*p^94 + 248049*p^93 + 268002*p^92 + 288141*p^91 + 309207*p^90 + 330264*p^89 + 352077*p^88 + 373653*p^87 + 395766*p^86 + 417363*p^85 + 439293*p^84 + 460431*p^83 + 481650*p^82 + 501798*p^81 + 521775*p^80 + 540402*p^79 + 558627*p^78 + 575235*p^77 + 591189*p^76 + 605289*p^75 + 618543*p^74 + 629718*p^73 + 639875*p^72 + 647756*p^71 + 654476*p^70 + 658830*p^69 + 661932*p^68 + 662577*p^67 + 661935*p^66 + 658827*p^65 + 654477*p^64 + 647757*p^63 + 639876*p^62 + 629718*p^61 + 618543*p^60 + 605283*p^59 + 591189*p^58 + 575235*p^57 + 558633*p^56 + 540402*p^55 + 521775*p^54 + 501798*p^53 + 481650*p^52 + 460431*p^51 + 439290*p^50 + 417363*p^49 + 395766*p^48 + 373656*p^47 + 352077*p^46 + 330264*p^45 + 309207*p^44 + 288141*p^43 + 268002*p^42 + 248052*p^41 + 229146*p^40 + 210627*p^39 + 193242*p^38 + 176334*p^37 + 160596*p^36 + 145422*p^35 + 131436*p^34 + 118071*p^33 + 105867*p^32 + 94284*p^31 + 83808*p^30 + 73953*p^29 + 65151*p^28 + 56943*p^27 + 49680*p^26 + 42957*p^25 + 37098*p^24 + 31725*p^23 + 27108*p^22 + 22896*p^21 + 19332*p^20 + 16119*p^19 + 13446*p^18 + 11043*p^17 + 9072*p^16 + 7317*p^15 + 5913*p^14 + 4671*p^13 + 3699*p^12 + 2835*p^11 + 2187*p^10 + 1620*p^9 + 1215*p^8 + 864*p^7 + 621*p^6 + 405*p^5 + 270*p^4 + 162*p^3 + 108*p^2 + 54*p + 27)/(p^134 + 2*p^133 + 4*p^132 + 6*p^131 + 10*p^130 + 15*p^129 + 23*p^128 + 32*p^127 + 45*p^126 + 60*p^125 + 81*p^124 + 105*p^123 + 137*p^122 + 173*p^121 + 219*p^120 + 271*p^119 + 336*p^118 + 409*p^117 + 498*p^116 + 597*p^115 + 716*p^114 + 848*p^113 + 1004*p^112 + 1175*p^111 + 1374*p^110 + 1591*p^109 + 1840*p^108 + 2109*p^107 + 2413*p^106 + 2739*p^105 + 3104*p^104 + 3492*p^103 + 3921*p^102 + 4373*p^101 + 4868*p^100 + 5386*p^99 + 5948*p^98 + 6531*p^97 + 7157*p^96 + 7801*p^95 + 8487*p^94 + 9187*p^93 + 9926*p^92 + 10672*p^91 + 11452*p^90 + 12232*p^89 + 13040*p^88 + 13839*p^87 + 14658*p^86 + 15458*p^85 + 16270*p^84 + 17053*p^83 + 17839*p^82 + 18585*p^81 + 19325*p^80 + 20015*p^79 + 20690*p^78 + 21305*p^77 + 21896*p^76 + 22418*p^75 + 22909*p^74 + 23323*p^73 + 23699*p^72 + 23991*p^71 + 24240*p^70 + 24401*p^69 + 24516*p^68 + 24540*p^67 + 24516*p^66 + 24401*p^65 + 24240*p^64 + 23991*p^63 + 23699*p^62 + 23323*p^61 + 22909*p^60 + 22418*p^59 + 21896*p^58 + 21305*p^57 + 20690*p^56 + 20015*p^55 + 19325*p^54 + 18585*p^53 + 17839*p^52 + 17053*p^51 + 16270*p^50 + 15458*p^49 + 14658*p^48 + 13839*p^47 + 13040*p^46 + 12232*p^45 + 11452*p^44 + 10672*p^43 + 9926*p^42 + 9187*p^41 + 8487*p^40 + 7801*p^39 + 7157*p^38 + 6531*p^37 + 5948*p^36 + 5386*p^35 + 4868*p^34 + 4373*p^33 + 3921*p^32 + 3492*p^31 + 3104*p^30 + 2739*p^29 + 2413*p^28 + 2109*p^27 + 1840*p^26 + 1591*p^25 + 1374*p^24 + 1175*p^23 + 1004*p^22 + 848*p^21 + 716*p^20 + 597*p^19 + 498*p^18 + 409*p^17 + 336*p^16 + 271*p^15 + 219*p^14 + 173*p^13 + 137*p^12 + 105*p^11 + 81*p^10 + 60*p^9 + 45*p^8 + 32*p^7 + 23*p^6 + 15*p^5 + 10*p^4 + 6*p^3 + 4*p^2 + 2*p + 1)
rho6 = 1/27*(27*p^183 + 81*p^182 + 189*p^181 + 324*p^180 + 540*p^179 + 837*p^178 + 1296*p^177 + 1917*p^176 + 2781*p^175 + 3888*p^174 + 5346*p^173 + 7182*p^172 + 9531*p^171 + 12420*p^170 + 16011*p^169 + 20358*p^168 + 25650*p^167 + 31941*p^166 + 39447*p^165 + 48249*p^164 + 58617*p^163 + 70659*p^162 + 84672*p^161 + 100764*p^160 + 119259*p^159 + 140319*p^158 + 164322*p^157 + 191457*p^156 + 222129*p^155 + 256554*p^154 + 295164*p^153 + 338175*p^152 + 386046*p^151 + 439020*p^150 + 497583*p^149 + 561978*p^148 + 632691*p^147 + 709938*p^146 + 794178*p^145 + 885600*p^144 + 984663*p^143 + 1091529*p^142 + 1206630*p^141 + 1330074*p^140 + 1462239*p^139 + 1603152*p^138 + 1753137*p^137 + 1912167*p^136 + 2080512*p^135 + 2258064*p^134 + 2445012*p^133 + 2641140*p^132 + 2846529*p^131 + 3060855*p^130 + 3284118*p^129 + 3515886*p^128 + 3756051*p^127 + 4004046*p^126 + 4259628*p^125 + 4522068*p^124 + 4790988*p^123 + 5065551*p^122 + 5345271*p^121 + 5629203*p^120 + 5916726*p^119 + 6206787*p^118 + 6498630*p^117 + 6791121*p^116 + 7083423*p^115 + 7374375*p^114 + 7663086*p^113 + 7948368*p^112 + 8229276*p^111 + 8504595*p^110 + 8773353*p^109 + 9034362*p^108 + 9286677*p^107 + 9529164*p^106 + 9760902*p^105 + 9980820*p^104 + 10188018*p^103 + 10381500*p^102 + 10560456*p^101 + 10724022*p^100 + 10871496*p^99 + 11002149*p^98 + 11115386*p^97 + 11210615*p^96 + 11287376*p^95 + 11345265*p^94 + 11384013*p^93 + 11403423*p^92 + 11403423*p^91 + 11384010*p^90 + 11345265*p^89 + 11287377*p^88 + 11210616*p^87 + 11115387*p^86 + 11002149*p^85 + 10871496*p^84 + 10724022*p^83 + 10560456*p^82 + 10381497*p^81 + 10188018*p^80 + 9980820*p^79 + 9760905*p^78 + 9529164*p^77 + 9286677*p^76 + 9034362*p^75 + 8773353*p^74 + 8504595*p^73 + 8229276*p^72 + 7948368*p^71 + 7663086*p^70 + 7374375*p^69 + 7083423*p^68 + 6791121*p^67 + 6498630*p^66 + 6206787*p^65 + 5916726*p^64 + 5629203*p^63 + 5345271*p^62 + 5065551*p^61 + 4790988*p^60 + 4522068*p^59 + 4259628*p^58 + 4004046*p^57 + 3756051*p^56 + 3515886*p^55 + 3284118*p^54 + 3060855*p^53 + 2846529*p^52 + 2641140*p^51 + 2445012*p^50 + 2258064*p^49 + 2080512*p^48 + 1912167*p^47 + 1753137*p^46 + 1603152*p^45 + 1462239*p^44 + 1330074*p^43 + 1206630*p^42 + 1091529*p^41 + 984663*p^40 + 885600*p^39 + 794178*p^38 + 709938*p^37 + 632691*p^36 + 561978*p^35 + 497583*p^34 + 439020*p^33 + 386046*p^32 + 338175*p^31 + 295164*p^30 + 256554*p^29 + 222129*p^28 + 191457*p^27 + 164322*p^26 + 140319*p^25 + 119259*p^24 + 100764*p^23 + 84672*p^22 + 70659*p^21 + 58617*p^20 + 48249*p^19 + 39447*p^18 + 31941*p^17 + 25650*p^16 + 20358*p^15 + 16011*p^14 + 12420*p^13 + 9531*p^12 + 7182*p^11 + 5346*p^10 + 3888*p^9 + 2781*p^8 + 1917*p^7 + 1296*p^6 + 837*p^5 + 540*p^4 + 324*p^3 + 189*p^2 + 81*p + 27)/(p^183 + 3*p^182 + 7*p^181 + 12*p^180 + 20*p^179 + 31*p^178 + 48*p^177 + 71*p^176 + 103*p^175 + 144*p^174 + 198*p^173 + 266*p^172 + 353*p^171 + 460*p^170 + 593*p^169 + 754*p^168 + 950*p^167 + 1183*p^166 + 1461*p^165 + 1787*p^164 + 2171*p^163 + 2617*p^162 + 3136*p^161 + 3732*p^160 + 4417*p^159 + 5197*p^158 + 6086*p^157 + 7091*p^156 + 8227*p^155 + 9502*p^154 + 10932*p^153 + 12525*p^152 + 14298*p^151 + 16260*p^150 + 18429*p^149 + 20814*p^148 + 23433*p^147 + 26294*p^146 + 29414*p^145 + 32800*p^144 + 36469*p^143 + 40427*p^142 + 44690*p^141 + 49262*p^140 + 54157*p^139 + 59376*p^138 + 64931*p^137 + 70821*p^136 + 77056*p^135 + 83632*p^134 + 90556*p^133 + 97820*p^132 + 105427*p^131 + 113365*p^130 + 121634*p^129 + 130218*p^128 + 139113*p^127 + 148298*p^126 + 157764*p^125 + 167484*p^124 + 177444*p^123 + 187613*p^122 + 197973*p^121 + 208489*p^120 + 219138*p^119 + 229881*p^118 + 240690*p^117 + 251523*p^116 + 262349*p^115 + 273125*p^114 + 283818*p^113 + 294384*p^112 + 304788*p^111 + 314985*p^110 + 324939*p^109 + 334606*p^108 + 343951*p^107 + 352932*p^106 + 361515*p^105 + 369660*p^104 + 377334*p^103 + 384500*p^102 + 391128*p^101 + 397186*p^100 + 402648*p^99 + 407487*p^98 + 411681*p^97 + 415208*p^96 + 418051*p^95 + 420195*p^94 + 421630*p^93 + 422349*p^92 + 422349*p^91 + 421630*p^90 + 420195*p^89 + 418051*p^88 + 415208*p^87 + 411681*p^86 + 407487*p^85 + 402648*p^84 + 397186*p^83 + 391128*p^82 + 384500*p^81 + 377334*p^80 + 369660*p^79 + 361515*p^78 + 352932*p^77 + 343951*p^76 + 334606*p^75 + 324939*p^74 + 314985*p^73 + 304788*p^72 + 294384*p^71 + 283818*p^70 + 273125*p^69 + 262349*p^68 + 251523*p^67 + 240690*p^66 + 229881*p^65 + 219138*p^64 + 208489*p^63 + 197973*p^62 + 187613*p^61 + 177444*p^60 + 167484*p^59 + 157764*p^58 + 148298*p^57 + 139113*p^56 + 130218*p^55 + 121634*p^54 + 113365*p^53 + 105427*p^52 + 97820*p^51 + 90556*p^50 + 83632*p^49 + 77056*p^48 + 70821*p^47 + 64931*p^46 + 59376*p^45 + 54157*p^44 + 49262*p^43 + 44690*p^42 + 40427*p^41 + 36469*p^40 + 32800*p^39 + 29414*p^38 + 26294*p^37 + 23433*p^36 + 20814*p^35 + 18429*p^34 + 16260*p^33 + 14298*p^32 + 12525*p^31 + 10932*p^30 + 9502*p^29 + 8227*p^28 + 7091*p^27 + 6086*p^26 + 5197*p^25 + 4417*p^24 + 3732*p^23 + 3136*p^22 + 2617*p^21 + 2171*p^20 + 1787*p^19 + 1461*p^18 + 1183*p^17 + 950*p^16 + 754*p^15 + 593*p^14 + 460*p^13 + 353*p^12 + 266*p^11 + 198*p^10 + 144*p^9 + 103*p^8 + 71*p^7 + 48*p^6 + 31*p^5 + 20*p^4 + 12*p^3 + 7*p^2 + 3*p + 1)
rho7 = 1/27*(27*p^270 + 81*p^269 + 189*p^268 + 324*p^267 + 540*p^266 + 837*p^265 + 1296*p^264 + 1917*p^263 + 2808*p^262 + 3969*p^261 + 5535*p^260 + 7506*p^259 + 10098*p^258 + 13338*p^257 + 17496*p^256 + 22572*p^255 + 28917*p^254 + 36558*p^253 + 45954*p^252 + 57132*p^251 + 70659*p^250 + 86589*p^249 + 105597*p^248 + 127737*p^247 + 153846*p^246 + 184032*p^245 + 219294*p^244 + 259713*p^243 + 306477*p^242 + 359721*p^241 + 420849*p^240 + 489996*p^239 + 568809*p^238 + 657477*p^237 + 757890*p^236 + 870237*p^235 + 996705*p^234 + 1137564*p^233 + 1295271*p^232 + 1470069*p^231 + 1664739*p^230 + 1879605*p^229 + 2117799*p^228 + 2379591*p^227 + 2668437*p^226 + 2984634*p^225 + 3331989*p^224 + 3710745*p^223 + 4125060*p^222 + 4575177*p^221 + 5065578*p^220 + 5596344*p^219 + 6172281*p^218 + 6793443*p^217 + 7464987*p^216 + 8186751*p^215 + 8964162*p^214 + 9796896*p^213 + 10690677*p^212 + 11644884*p^211 + 12665511*p^210 + 13751721*p^209 + 14909724*p^208 + 16138251*p^207 + 17443674*p^206 + 18824400*p^205 + 20286990*p^204 + 21829311*p^203 + 23458005*p^202 + 25170453*p^201 + 26973378*p^200 + 28863486*p^199 + 30847500*p^198 + 32921505*p^197 + 35092197*p^196 + 37354797*p^195 + 39715866*p^194 + 42169896*p^193 + 44723340*p^192 + 47369718*p^191 + 50115213*p^190 + 52952454*p^189 + 55887435*p^188 + 58911705*p^187 + 62030961*p^186 + 65235780*p^185 + 68531589*p^184 + 71907804*p^183 + 75369501*p^182 + 78905124*p^181 + 82519506*p^180 + 86199930*p^179 + 89950851*p^178 + 93758553*p^177 + 97627248*p^176 + 101542140*p^175 + 105507144*p^174 + 109506546*p^173 + 113544072*p^172 + 117602955*p^171 + 121686678*p^170 + 125777691*p^169 + 129879423*p^168 + 133973460*p^167 + 138063069*p^166 + 142129188*p^165 + 146175138*p^164 + 150181182*p^163 + 154150668*p^162 + 158063454*p^161 + 161923104*p^160 + 165709017*p^159 + 169424946*p^158 + 173050155*p^157 + 176588856*p^156 + 180020151*p^155 + 183348657*p^154 + 186553611*p^153 + 189640278*p^152 + 192588057*p^151 + 195402834*p^150 + 198064440*p^149 + 200579598*p^148 + 202928544*p^147 + 205118811*p^146 + 207131337*p^145 + 208974681*p^144 + 210630429*p^143 + 212108031*p^142 + 213389936*p^141 + 214486704*p^140 + 215381619*p^139 + 216086238*p^138 + 216584874*p^137 + 216890190*p^136 + 216987417*p^135 + 216890190*p^134 + 216584874*p^133 + 216086238*p^132 + 215381619*p^131 + 214486704*p^130 + 213389937*p^129 + 212108031*p^128 + 210630429*p^127 + 208974681*p^126 + 207131337*p^125 + 205118811*p^124 + 202928544*p^123 + 200579598*p^122 + 198064440*p^121 + 195402834*p^120 + 192588057*p^119 + 189640278*p^118 + 186553611*p^117 + 183348657*p^116 + 180020151*p^115 + 176588856*p^114 + 173050155*p^113 + 169424946*p^112 + 165709017*p^111 + 161923104*p^110 + 158063454*p^109 + 154150668*p^108 + 150181182*p^107 + 146175138*p^106 + 142129188*p^105 + 138063069*p^104 + 133973460*p^103 + 129879423*p^102 + 125777691*p^101 + 121686678*p^100 + 117602955*p^99 + 113544072*p^98 + 109506546*p^97 + 105507144*p^96 + 101542140*p^95 + 97627248*p^94 + 93758553*p^93 + 89950851*p^92 + 86199930*p^91 + 82519506*p^90 + 78905124*p^89 + 75369501*p^88 + 71907804*p^87 + 68531589*p^86 + 65235780*p^85 + 62030961*p^84 + 58911705*p^83 + 55887435*p^82 + 52952454*p^81 + 50115213*p^80 + 47369718*p^79 + 44723340*p^78 + 42169896*p^77 + 39715866*p^76 + 37354797*p^75 + 35092197*p^74 + 32921505*p^73 + 30847500*p^72 + 28863486*p^71 + 26973378*p^70 + 25170453*p^69 + 23458005*p^68 + 21829311*p^67 + 20286990*p^66 + 18824400*p^65 + 17443674*p^64 + 16138251*p^63 + 14909724*p^62 + 13751721*p^61 + 12665511*p^60 + 11644884*p^59 + 10690677*p^58 + 9796896*p^57 + 8964162*p^56 + 8186751*p^55 + 7464987*p^54 + 6793443*p^53 + 6172281*p^52 + 5596344*p^51 + 5065578*p^50 + 4575177*p^49 + 4125060*p^48 + 3710745*p^47 + 3331989*p^46 + 2984634*p^45 + 2668437*p^44 + 2379591*p^43 + 2117799*p^42 + 1879605*p^41 + 1664739*p^40 + 1470069*p^39 + 1295271*p^38 + 1137564*p^37 + 996705*p^36 + 870237*p^35 + 757890*p^34 + 657477*p^33 + 568809*p^32 + 489996*p^31 + 420849*p^30 + 359721*p^29 + 306477*p^28 + 259713*p^27 + 219294*p^26 + 184032*p^25 + 153846*p^24 + 127737*p^23 + 105597*p^22 + 86589*p^21 + 70659*p^20 + 57132*p^19 + 45954*p^18 + 36558*p^17 + 28917*p^16 + 22572*p^15 + 17496*p^14 + 13338*p^13 + 10098*p^12 + 7506*p^11 + 5535*p^10 + 3969*p^9 + 2808*p^8 + 1917*p^7 + 1296*p^6 + 837*p^5 + 540*p^4 + 324*p^3 + 189*p^2 + 81*p + 27)/(p^270 + 3*p^269 + 7*p^268 + 12*p^267 + 20*p^266 + 31*p^265 + 48*p^264 + 71*p^263 + 104*p^262 + 147*p^261 + 205*p^260 + 278*p^259 + 374*p^258 + 494*p^257 + 648*p^256 + 836*p^255 + 1071*p^254 + 1354*p^253 + 1702*p^252 + 2116*p^251 + 2617*p^250 + 3207*p^249 + 3911*p^248 + 4731*p^247 + 5698*p^246 + 6816*p^245 + 8122*p^244 + 9619*p^243 + 11351*p^242 + 13323*p^241 + 15587*p^240 + 18148*p^239 + 21067*p^238 + 24351*p^237 + 28070*p^236 + 32231*p^235 + 36915*p^234 + 42132*p^233 + 47973*p^232 + 54447*p^231 + 61657*p^230 + 69615*p^229 + 78437*p^228 + 88133*p^227 + 98831*p^226 + 110542*p^225 + 123407*p^224 + 137435*p^223 + 152780*p^222 + 169451*p^221 + 187614*p^220 + 207272*p^219 + 228603*p^218 + 251609*p^217 + 276481*p^216 + 303213*p^215 + 332006*p^214 + 362848*p^213 + 395951*p^212 + 431292*p^211 + 469093*p^210 + 509323*p^209 + 552212*p^208 + 597713*p^207 + 646062*p^206 + 697200*p^205 + 751370*p^204 + 808493*p^203 + 868815*p^202 + 932239*p^201 + 999014*p^200 + 1069018*p^199 + 1142500*p^198 + 1219315*p^197 + 1299711*p^196 + 1383511*p^195 + 1470958*p^194 + 1561848*p^193 + 1656420*p^192 + 1754434*p^191 + 1856119*p^190 + 1961202*p^189 + 2069905*p^188 + 2181915*p^187 + 2297443*p^186 + 2416140*p^185 + 2538207*p^184 + 2663252*p^183 + 2791463*p^182 + 2922412*p^181 + 3056278*p^180 + 3192590*p^179 + 3331513*p^178 + 3472539*p^177 + 3615824*p^176 + 3760820*p^175 + 3907672*p^174 + 4055798*p^173 + 4205336*p^172 + 4355665*p^171 + 4506914*p^170 + 4658433*p^169 + 4810349*p^168 + 4961980*p^167 + 5113447*p^166 + 5264044*p^165 + 5413894*p^164 + 5562266*p^163 + 5709284*p^162 + 5854202*p^161 + 5997152*p^160 + 6137371*p^159 + 6274998*p^158 + 6409265*p^157 + 6540328*p^156 + 6667413*p^155 + 6790691*p^154 + 6909393*p^153 + 7023714*p^152 + 7132891*p^151 + 7237142*p^150 + 7335720*p^149 + 7428874*p^148 + 7515872*p^147 + 7596993*p^146 + 7671531*p^145 + 7739803*p^144 + 7801127*p^143 + 7855853*p^142 + 7903331*p^141 + 7943952*p^140 + 7977097*p^139 + 8003194*p^138 + 8021662*p^137 + 8032970*p^136 + 8036571*p^135 + 8032970*p^134 + 8021662*p^133 + 8003194*p^132 + 7977097*p^131 + 7943952*p^130 + 7903331*p^129 + 7855853*p^128 + 7801127*p^127 + 7739803*p^126 + 7671531*p^125 + 7596993*p^124 + 7515872*p^123 + 7428874*p^122 + 7335720*p^121 + 7237142*p^120 + 7132891*p^119 + 7023714*p^118 + 6909393*p^117 + 6790691*p^116 + 6667413*p^115 + 6540328*p^114 + 6409265*p^113 + 6274998*p^112 + 6137371*p^111 + 5997152*p^110 + 5854202*p^109 + 5709284*p^108 + 5562266*p^107 + 5413894*p^106 + 5264044*p^105 + 5113447*p^104 + 4961980*p^103 + 4810349*p^102 + 4658433*p^101 + 4506914*p^100 + 4355665*p^99 + 4205336*p^98 + 4055798*p^97 + 3907672*p^96 + 3760820*p^95 + 3615824*p^94 + 3472539*p^93 + 3331513*p^92 + 3192590*p^91 + 3056278*p^90 + 2922412*p^89 + 2791463*p^88 + 2663252*p^87 + 2538207*p^86 + 2416140*p^85 + 2297443*p^84 + 2181915*p^83 + 2069905*p^82 + 1961202*p^81 + 1856119*p^80 + 1754434*p^79 + 1656420*p^78 + 1561848*p^77 + 1470958*p^76 + 1383511*p^75 + 1299711*p^74 + 1219315*p^73 + 1142500*p^72 + 1069018*p^71 + 999014*p^70 + 932239*p^69 + 868815*p^68 + 808493*p^67 + 751370*p^66 + 697200*p^65 + 646062*p^64 + 597713*p^63 + 552212*p^62 + 509323*p^61 + 469093*p^60 + 431292*p^59 + 395951*p^58 + 362848*p^57 + 332006*p^56 + 303213*p^55 + 276481*p^54 + 251609*p^53 + 228603*p^52 + 207272*p^51 + 187614*p^50 + 169451*p^49 + 152780*p^48 + 137435*p^47 + 123407*p^46 + 110542*p^45 + 98831*p^44 + 88133*p^43 + 78437*p^42 + 69615*p^41 + 61657*p^40 + 54447*p^39 + 47973*p^38 + 42132*p^37 + 36915*p^36 + 32231*p^35 + 28070*p^34 + 24351*p^33 + 21067*p^32 + 18148*p^31 + 15587*p^30 + 13323*p^29 + 11351*p^28 + 9619*p^27 + 8122*p^26 + 6816*p^25 + 5698*p^24 + 4731*p^23 + 3911*p^22 + 3207*p^21 + 2617*p^20 + 2116*p^19 + 1702*p^18 + 1354*p^17 + 1071*p^16 + 836*p^15 + 648*p^14 + 494*p^13 + 374*p^12 + 278*p^11 + 205*p^10 + 147*p^9 + 104*p^8 + 71*p^7 + 48*p^6 + 31*p^5 + 20*p^4 + 12*p^3 + 7*p^2 + 3*p + 1)
rho8 = 1/27*(27*p^420 + 108*p^419 + 297*p^418 + 594*p^417 + 1053*p^416 + 1701*p^415 + 2673*p^414 + 4050*p^413 + 6021*p^412 + 8721*p^411 + 12420*p^410 + 17307*p^409 + 23733*p^408 + 31995*p^407 + 42633*p^406 + 56106*p^405 + 73143*p^404 + 94365*p^403 + 120744*p^402 + 153117*p^401 + 192753*p^400 + 240786*p^399 + 298890*p^398 + 368577*p^397 + 451980*p^396 + 551070*p^395 + 668547*p^394 + 806922*p^393 + 969543*p^392 + 1159596*p^391 + 1381266*p^390 + 1638549*p^389 + 1936575*p^388 + 2280285*p^387 + 2675943*p^386 + 3129597*p^385 + 3648807*p^384 + 4240944*p^383 + 4915107*p^382 + 5680179*p^381 + 6546987*p^380 + 7526196*p^379 + 8630685*p^378 + 9873144*p^377 + 11268720*p^376 + 12832425*p^375 + 14582052*p^374 + 16535232*p^373 + 18712647*p^372 + 21134871*p^371 + 23825934*p^370 + 26809758*p^369 + 30114018*p^368 + 33766281*p^367 + 37798272*p^366 + 42241608*p^365 + 47132442*p^364 + 52506846*p^363 + 58405887*p^362 + 64870497*p^361 + 71946981*p^360 + 79681482*p^359 + 88126002*p^358 + 97332327*p^357 + 107358534*p^356 + 118262457*p^355 + 130108707*p^354 + 142961517*p^353 + 156892302*p^352 + 171972018*p^351 + 188279343*p^350 + 205892334*p^349 + 224897202*p^348 + 245379348*p^347 + 267432840*p^346 + 291150666*p^345 + 316634913*p^344 + 343986345*p^343 + 373315338*p^342 + 404730594*p^341 + 438350805*p^340 + 474292638*p^339 + 512683209*p^338 + 553647123*p^337 + 597319758*p^336 + 643833522*p^335 + 693332055*p^334 + 745955406*p^333 + 801855099*p^332 + 861178392*p^331 + 924084396*p^330 + 990727173*p^329 + 1061272827*p^328 + 1135881576*p^327 + 1214725950*p^326 + 1297971648*p^325 + 1385796762*p^324 + 1478371554*p^323 + 1575878814*p^322 + 1678492368*p^321 + 1786398543*p^320 + 1899773541*p^319 + 2018806119*p^318 + 2143673586*p^317 + 2274565671*p^316 + 2411659305*p^315 + 2555143812*p^314 + 2705194233*p^313 + 2861997759*p^312 + 3025725786*p^311 + 3196561725*p^310 + 3374671572*p^309 + 3560233014*p^308 + 3753404703*p^307 + 3954356793*p^306 + 4163238648*p^305 + 4380210729*p^304 + 4605410952*p^303 + 4838988087*p^302 + 5081066523*p^301 + 5331781098*p^300 + 5591240487*p^299 + 5859563571*p^298 + 6136841151*p^297 + 6423173775*p^296 + 6718632075*p^295 + 7023296241*p^294 + 7337214684*p^293 + 7660444995*p^292 + 7993011150*p^291 + 8334946170*p^290 + 8686247625*p^289 + 9046921860*p^288 + 9416938014*p^287 + 9796274001*p^286 + 10184868828*p^285 + 10582670169*p^284 + 10989585198*p^283 + 11405529918*p^282 + 11830378293*p^281 + 12264013224*p^280 + 12706274223*p^279 + 13157010198*p^278 + 13616025372*p^277 + 14083133742*p^276 + 14558103513*p^275 + 15040713420*p^274 + 15530695407*p^273 + 16027792650*p^272 + 16531700778*p^271 + 17042127732*p^270 + 17558733258*p^269 + 18081190413*p^268 + 18609123654*p^267 + 19142172153*p^266 + 19679926212*p^265 + 20221992198*p^264 + 20767927581*p^263 + 21317307651*p^262 + 21869658882*p^261 + 22424527242*p^260 + 22981410207*p^259 + 23539826853*p^258 + 24099248223*p^257 + 24659168931*p^256 + 25219036233*p^255 + 25778323332*p^254 + 26336456910*p^253 + 26892891837*p^252 + 27447037488*p^251 + 27998334045*p^250 + 28546177329*p^249 + 29089996425*p^248 + 29629177353*p^247 + 30163142205*p^246 + 30691271412*p^245 + 31212984123*p^244 + 31727659392*p^243 + 32234717907*p^242 + 32733541827*p^241 + 33223557618*p^240 + 33704154945*p^239 + 34174770750*p^238 + 34634806902*p^237 + 35083715220*p^236 + 35520914259*p^235 + 35945875413*p^234 + 36358038540*p^233 + 36756898848*p^232 + 37141921791*p^231 + 37512630900*p^230 + 37868521653*p^229 + 38209149873*p^228 + 38534044977*p^227 + 38842799130*p^226 + 39134979603*p^225 + 39410218332*p^224 + 39668123790*p^223 + 39908371167*p^222 + 40130613459*p^221 + 40334571891*p^220 + 40519946600*p^219 + 40686507549*p^218 + 40834004553*p^217 + 40962258306*p^216 + 41071070142*p^215 + 41160313350*p^214 + 41229842454*p^213 + 41279584446*p^212 + 41309447958*p^211 + 41319414630*p^210 + 41309447958*p^209 + 41279584446*p^208 + 41229842454*p^207 + 41160313350*p^206 + 41071070142*p^205 + 40962258306*p^204 + 40834004553*p^203 + 40686507549*p^202 + 40519946601*p^201 + 40334571891*p^200 + 40130613459*p^199 + 39908371167*p^198 + 39668123790*p^197 + 39410218332*p^196 + 39134979603*p^195 + 38842799130*p^194 + 38534044977*p^193 + 38209149873*p^192 + 37868521653*p^191 + 37512630900*p^190 + 37141921791*p^189 + 36756898848*p^188 + 36358038540*p^187 + 35945875413*p^186 + 35520914259*p^185 + 35083715220*p^184 + 34634806902*p^183 + 34174770750*p^182 + 33704154945*p^181 + 33223557618*p^180 + 32733541827*p^179 + 32234717907*p^178 + 31727659392*p^177 + 31212984123*p^176 + 30691271412*p^175 + 30163142205*p^174 + 29629177353*p^173 + 29089996425*p^172 + 28546177329*p^171 + 27998334045*p^170 + 27447037488*p^169 + 26892891837*p^168 + 26336456910*p^167 + 25778323332*p^166 + 25219036233*p^165 + 24659168931*p^164 + 24099248223*p^163 + 23539826853*p^162 + 22981410207*p^161 + 22424527242*p^160 + 21869658882*p^159 + 21317307651*p^158 + 20767927581*p^157 + 20221992198*p^156 + 19679926212*p^155 + 19142172153*p^154 + 18609123654*p^153 + 18081190413*p^152 + 17558733258*p^151 + 17042127732*p^150 + 16531700778*p^149 + 16027792650*p^148 + 15530695407*p^147 + 15040713420*p^146 + 14558103513*p^145 + 14083133742*p^144 + 13616025372*p^143 + 13157010198*p^142 + 12706274223*p^141 + 12264013224*p^140 + 11830378293*p^139 + 11405529918*p^138 + 10989585198*p^137 + 10582670169*p^136 + 10184868828*p^135 + 9796274001*p^134 + 9416938014*p^133 + 9046921860*p^132 + 8686247625*p^131 + 8334946170*p^130 + 7993011150*p^129 + 7660444995*p^128 + 7337214684*p^127 + 7023296241*p^126 + 6718632075*p^125 + 6423173775*p^124 + 6136841151*p^123 + 5859563571*p^122 + 5591240487*p^121 + 5331781098*p^120 + 5081066523*p^119 + 4838988087*p^118 + 4605410952*p^117 + 4380210729*p^116 + 4163238648*p^115 + 3954356793*p^114 + 3753404703*p^113 + 3560233014*p^112 + 3374671572*p^111 + 3196561725*p^110 + 3025725786*p^109 + 2861997759*p^108 + 2705194233*p^107 + 2555143812*p^106 + 2411659305*p^105 + 2274565671*p^104 + 2143673586*p^103 + 2018806119*p^102 + 1899773541*p^101 + 1786398543*p^100 + 1678492368*p^99 + 1575878814*p^98 + 1478371554*p^97 + 1385796762*p^96 + 1297971648*p^95 + 1214725950*p^94 + 1135881576*p^93 + 1061272827*p^92 + 990727173*p^91 + 924084396*p^90 + 861178392*p^89 + 801855099*p^88 + 745955406*p^87 + 693332055*p^86 + 643833522*p^85 + 597319758*p^84 + 553647123*p^83 + 512683209*p^82 + 474292638*p^81 + 438350805*p^80 + 404730594*p^79 + 373315338*p^78 + 343986345*p^77 + 316634913*p^76 + 291150666*p^75 + 267432840*p^74 + 245379348*p^73 + 224897202*p^72 + 205892334*p^71 + 188279343*p^70 + 171972018*p^69 + 156892302*p^68 + 142961517*p^67 + 130108707*p^66 + 118262457*p^65 + 107358534*p^64 + 97332327*p^63 + 88126002*p^62 + 79681482*p^61 + 71946981*p^60 + 64870497*p^59 + 58405887*p^58 + 52506846*p^57 + 47132442*p^56 + 42241608*p^55 + 37798272*p^54 + 33766281*p^53 + 30114018*p^52 + 26809758*p^51 + 23825934*p^50 + 21134871*p^49 + 18712647*p^48 + 16535232*p^47 + 14582052*p^46 + 12832425*p^45 + 11268720*p^44 + 9873144*p^43 + 8630685*p^42 + 7526196*p^41 + 6546987*p^40 + 5680179*p^39 + 4915107*p^38 + 4240944*p^37 + 3648807*p^36 + 3129597*p^35 + 2675943*p^34 + 2280285*p^33 + 1936575*p^32 + 1638549*p^31 + 1381266*p^30 + 1159596*p^29 + 969543*p^28 + 806922*p^27 + 668547*p^26 + 551070*p^25 + 451980*p^24 + 368577*p^23 + 298890*p^22 + 240786*p^21 + 192753*p^20 + 153117*p^19 + 120744*p^18 + 94365*p^17 + 73143*p^16 + 56106*p^15 + 42633*p^14 + 31995*p^13 + 23733*p^12 + 17307*p^11 + 12420*p^10 + 8721*p^9 + 6021*p^8 + 4050*p^7 + 2673*p^6 + 1701*p^5 + 1053*p^4 + 594*p^3 + 297*p^2 + 108*p + 27)/(p^420 + 4*p^419 + 11*p^418 + 22*p^417 + 39*p^416 + 63*p^415 + 99*p^414 + 150*p^413 + 223*p^412 + 323*p^411 + 460*p^410 + 641*p^409 + 879*p^408 + 1185*p^407 + 1579*p^406 + 2078*p^405 + 2709*p^404 + 3495*p^403 + 4472*p^402 + 5671*p^401 + 7139*p^400 + 8918*p^399 + 11070*p^398 + 13651*p^397 + 16740*p^396 + 20410*p^395 + 24761*p^394 + 29886*p^393 + 35909*p^392 + 42948*p^391 + 51158*p^390 + 60687*p^389 + 71725*p^388 + 84455*p^387 + 99109*p^386 + 115911*p^385 + 135141*p^384 + 157072*p^383 + 182041*p^382 + 210377*p^381 + 242481*p^380 + 278748*p^379 + 319655*p^378 + 365672*p^377 + 417360*p^376 + 475275*p^375 + 540076*p^374 + 612416*p^373 + 693061*p^372 + 782773*p^371 + 882442*p^370 + 992954*p^369 + 1115334*p^368 + 1250603*p^367 + 1399936*p^366 + 1564504*p^365 + 1745646*p^364 + 1944698*p^363 + 2163181*p^362 + 2402611*p^361 + 2664703*p^360 + 2951166*p^359 + 3263926*p^358 + 3604901*p^357 + 3976242*p^356 + 4380091*p^355 + 4818841*p^354 + 5294871*p^353 + 5810826*p^352 + 6369334*p^351 + 6973309*p^350 + 7625642*p^349 + 8329526*p^348 + 9088124*p^347 + 9904920*p^346 + 10783358*p^345 + 11727219*p^344 + 12740235*p^343 + 13826494*p^342 + 14990022*p^341 + 16235215*p^340 + 17566394*p^339 + 18988267*p^338 + 20505449*p^337 + 22122954*p^336 + 23845686*p^335 + 25678965*p^334 + 27627978*p^333 + 29698337*p^332 + 31895496*p^331 + 34225348*p^330 + 36693599*p^329 + 39306401*p^328 + 42069688*p^327 + 44989850*p^326 + 48073024*p^325 + 51325806*p^324 + 54754502*p^323 + 58365882*p^322 + 62166384*p^321 + 66162909*p^320 + 70361983*p^319 + 74770597*p^318 + 79395318*p^317 + 84243173*p^316 + 89320715*p^315 + 94634956*p^314 + 100192379*p^313 + 105999917*p^312 + 112063918*p^311 + 118391175*p^310 + 124987836*p^309 + 131860482*p^308 + 139014989*p^307 + 146457659*p^306 + 154194024*p^305 + 162230027*p^304 + 170570776*p^303 + 179221781*p^302 + 188187649*p^301 + 197473374*p^300 + 207082981*p^299 + 217020873*p^298 + 227290413*p^297 + 237895325*p^296 + 248838225*p^295 + 260122083*p^294 + 271748692*p^293 + 283720185*p^292 + 296037450*p^291 + 308701710*p^290 + 321712875*p^289 + 335071180*p^288 + 348775482*p^287 + 362824963*p^286 + 377217364*p^285 + 391950747*p^284 + 407021674*p^283 + 422427034*p^282 + 438162159*p^281 + 454222712*p^280 + 470602749*p^279 + 487296674*p^278 + 504297236*p^277 + 521597546*p^276 + 539189019*p^275 + 557063460*p^274 + 575210941*p^273 + 593621950*p^272 + 612285214*p^271 + 631189916*p^270 + 650323454*p^269 + 669673719*p^268 + 689226802*p^267 + 708969339*p^266 + 728886156*p^265 + 748962674*p^264 + 769182503*p^263 + 789529913*p^262 + 809987366*p^261 + 830538046*p^260 + 851163341*p^259 + 871845439*p^258 + 892564749*p^257 + 913302553*p^256 + 934038379*p^255 + 954752716*p^254 + 975424330*p^253 + 996033031*p^252 + 1016556944*p^251 + 1036975335*p^250 + 1057265827*p^249 + 1077407275*p^248 + 1097376939*p^247 + 1117153415*p^246 + 1136713756*p^245 + 1156036449*p^244 + 1175098496*p^243 + 1193878441*p^242 + 1212353401*p^241 + 1230502134*p^240 + 1248302035*p^239 + 1265732250*p^238 + 1282770626*p^237 + 1299396860*p^236 + 1315589417*p^235 + 1331328719*p^234 + 1346594020*p^233 + 1361366624*p^232 + 1375626733*p^231 + 1389356700*p^230 + 1402537839*p^229 + 1415153699*p^228 + 1427186851*p^227 + 1438622190*p^226 + 1449443689*p^225 + 1459637716*p^224 + 1469189770*p^223 + 1478087821*p^222 + 1486319017*p^221 + 1493873033*p^220 + 1500738763*p^219 + 1506907687*p^218 + 1512370539*p^217 + 1517120678*p^216 + 1521150746*p^215 + 1524456050*p^214 + 1527031202*p^213 + 1528873498*p^212 + 1529979554*p^211 + 1530348690*p^210 + 1529979554*p^209 + 1528873498*p^208 + 1527031202*p^207 + 1524456050*p^206 + 1521150746*p^205 + 1517120678*p^204 + 1512370539*p^203 + 1506907687*p^202 + 1500738763*p^201 + 1493873033*p^200 + 1486319017*p^199 + 1478087821*p^198 + 1469189770*p^197 + 1459637716*p^196 + 1449443689*p^195 + 1438622190*p^194 + 1427186851*p^193 + 1415153699*p^192 + 1402537839*p^191 + 1389356700*p^190 + 1375626733*p^189 + 1361366624*p^188 + 1346594020*p^187 + 1331328719*p^186 + 1315589417*p^185 + 1299396860*p^184 + 1282770626*p^183 + 1265732250*p^182 + 1248302035*p^181 + 1230502134*p^180 + 1212353401*p^179 + 1193878441*p^178 + 1175098496*p^177 + 1156036449*p^176 + 1136713756*p^175 + 1117153415*p^174 + 1097376939*p^173 + 1077407275*p^172 + 1057265827*p^171 + 1036975335*p^170 + 1016556944*p^169 + 996033031*p^168 + 975424330*p^167 + 954752716*p^166 + 934038379*p^165 + 913302553*p^164 + 892564749*p^163 + 871845439*p^162 + 851163341*p^161 + 830538046*p^160 + 809987366*p^159 + 789529913*p^158 + 769182503*p^157 + 748962674*p^156 + 728886156*p^155 + 708969339*p^154 + 689226802*p^153 + 669673719*p^152 + 650323454*p^151 + 631189916*p^150 + 612285214*p^149 + 593621950*p^148 + 575210941*p^147 + 557063460*p^146 + 539189019*p^145 + 521597546*p^144 + 504297236*p^143 + 487296674*p^142 + 470602749*p^141 + 454222712*p^140 + 438162159*p^139 + 422427034*p^138 + 407021674*p^137 + 391950747*p^136 + 377217364*p^135 + 362824963*p^134 + 348775482*p^133 + 335071180*p^132 + 321712875*p^131 + 308701710*p^130 + 296037450*p^129 + 283720185*p^128 + 271748692*p^127 + 260122083*p^126 + 248838225*p^125 + 237895325*p^124 + 227290413*p^123 + 217020873*p^122 + 207082981*p^121 + 197473374*p^120 + 188187649*p^119 + 179221781*p^118 + 170570776*p^117 + 162230027*p^116 + 154194024*p^115 + 146457659*p^114 + 139014989*p^113 + 131860482*p^112 + 124987836*p^111 + 118391175*p^110 + 112063918*p^109 + 105999917*p^108 + 100192379*p^107 + 94634956*p^106 + 89320715*p^105 + 84243173*p^104 + 79395318*p^103 + 74770597*p^102 + 70361983*p^101 + 66162909*p^100 + 62166384*p^99 + 58365882*p^98 + 54754502*p^97 + 51325806*p^96 + 48073024*p^95 + 44989850*p^94 + 42069688*p^93 + 39306401*p^92 + 36693599*p^91 + 34225348*p^90 + 31895496*p^89 + 29698337*p^88 + 27627978*p^87 + 25678965*p^86 + 23845686*p^85 + 22122954*p^84 + 20505449*p^83 + 18988267*p^82 + 17566394*p^81 + 16235215*p^80 + 14990022*p^79 + 13826494*p^78 + 12740235*p^77 + 11727219*p^76 + 10783358*p^75 + 9904920*p^74 + 9088124*p^73 + 8329526*p^72 + 7625642*p^71 + 6973309*p^70 + 6369334*p^69 + 5810826*p^68 + 5294871*p^67 + 4818841*p^66 + 4380091*p^65 + 3976242*p^64 + 3604901*p^63 + 3263926*p^62 + 2951166*p^61 + 2664703*p^60 + 2402611*p^59 + 2163181*p^58 + 1944698*p^57 + 1745646*p^56 + 1564504*p^55 + 1399936*p^54 + 1250603*p^53 + 1115334*p^52 + 992954*p^51 + 882442*p^50 + 782773*p^49 + 693061*p^48 + 612416*p^47 + 540076*p^46 + 475275*p^45 + 417360*p^44 + 365672*p^43 + 319655*p^42 + 278748*p^41 + 242481*p^40 + 210377*p^39 + 182041*p^38 + 157072*p^37 + 135141*p^36 + 115911*p^35 + 99109*p^34 + 84455*p^33 + 71725*p^32 + 60687*p^31 + 51158*p^30 + 42948*p^29 + 35909*p^28 + 29886*p^27 + 24761*p^26 + 20410*p^25 + 16740*p^24 + 13651*p^23 + 11070*p^22 + 8918*p^21 + 7139*p^20 + 5671*p^19 + 4472*p^18 + 3495*p^17 + 2709*p^16 + 2078*p^15 + 1579*p^14 + 1185*p^13 + 879*p^12 + 641*p^11 + 460*p^10 + 323*p^9 + 223*p^8 + 150*p^7 + 99*p^6 + 63*p^5 + 39*p^4 + 22*p^3 + 11*p^2 + 4*p + 1)
rho9 = 1

# time elapsed in seconds (computed 18 March 2024)
time1 = 0.118717908859
time2 = 0.162343978882
time3 = 0.433195829391
time4 = 2.59619402885
time5 = 14.0321698189
time6 = 67.1236462593
time7 = 214.628448963
time8 = 563.739546061
time9 = 7.69453501701

# For reference: old computing times - all 64 equations set up and solved at once (issues with n=8,9)
time2_old = 0.341377019882
time3_old = 0.657305955887
time4_old = 3.74625587463
time5_old = 24.279
time6_old = 174.853693008
time7_old = 609.009339094